# PARTE 3 E 4: REGRESSÃO LOGÍSTICA E SVM

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# PARTE 3: Regressão Logística

1. Utilize a base de dados construída no Trabalho 3 ‘risco_credito.pkl’, que possui 14 registros, para testar o algoritmo de Regressão Logística.

2. Faça o Encoder dos dados e, para facilitar, como fizemos na aula teórica, apague os registros que possuem a classe ‘moderado’. No total teremos 11 registros.

3. Treine o algoritmo de regressão logística e utilize o parâmetro ‘random_state =1’ para ter sempre o mesmo resultado.

4. Utilize o comando ‘.intercept_’ para ter o resultado do B0.
O resultado deve ser =-0.80828993

5. Utilize o comando ‘.coef_’ para ter o resultado dos demais parâmetros que deve ser:
array([[-0.76704533,  0.23906678, -0.47976059,  1.12186218]])

6. Agora utilize o comando ‘predict’ para fazer o teste do seu algoritmo com:

    a) história boa, dívida alta, garantias nenhuma, renda > 35
    (o resultado desse teste deve ser ‘baixo’)

    b) história ruim, dívida alta, garantias adequada, renda < 15
    (o resultado desse teste deve ser ‘alto’)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import pickle

## 1. Carregar a base de dados risco_credito.pkl

In [21]:
# Carregar o dataset original para obter os labels
dataset_risco_credito = pd.read_csv('dataset_risco_credito.csv')
print("Dataset original:")
print(dataset_risco_credito)
print(f"\nTotal de registros: {len(dataset_risco_credito)}")

Dataset original:
        historia divida garantias     renda     risco
0           ruim   alta   nenhuma      0_15      alto
1   desconhecida   alta   nenhuma     15_35      alto
2   desconhecida  baixa   nenhuma     15_35  moderado
3   desconhecida  baixa   nenhuma  acima_35      alto
4   desconhecida  baixa   nenhuma  acima_35     baixo
5   desconhecida  baixa  adequada  acima_35     baixo
6           ruim  baixa   nenhuma      0_15      alto
7           ruim  baixa  adequada  acima_35  moderado
8            boa  baixa   nenhuma  acima_35     baixo
9            boa   alta  adequada  acima_35     baixo
10           boa   alta   nenhuma      0_15      alto
11           boa   alta   nenhuma     15_35  moderado
12           boa   alta   nenhuma  acima_35     baixo
13          ruim   alta   nenhuma     15_35      alto

Total de registros: 14


## 2. Remover registros com classe 'moderado' e fazer o encoding

In [22]:
# Filtrar dados removendo a classe 'moderado'
dataset_filtrado = dataset_risco_credito[dataset_risco_credito['risco'] != 'moderado'].copy()
print(f"\nDataset após remover 'moderado':")
print(dataset_filtrado)
print(f"\nTotal de registros: {len(dataset_filtrado)}")

# Separar X e y
X_risco_credito_filtrado = dataset_filtrado.iloc[:, 0:-1].values
y_risco_credito_filtrado = dataset_filtrado.iloc[:, -1].values

# Fazer encoding das features (X)
X_encoded = X_risco_credito_filtrado.copy()
label_encoders = []

for i in range(X_encoded.shape[1]):
    le = LabelEncoder()
    X_encoded[:, i] = le.fit_transform(X_encoded[:, i])
    label_encoders.append(le)

# Fazer encoding da classe (y)
le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y_risco_credito_filtrado)

print("\nX codificado:")
print(X_encoded)
print("\ny codificado:")
print(y_encoded)
print(f"\nClasses: {le_y.classes_}")


Dataset após remover 'moderado':
        historia divida garantias     renda  risco
0           ruim   alta   nenhuma      0_15   alto
1   desconhecida   alta   nenhuma     15_35   alto
3   desconhecida  baixa   nenhuma  acima_35   alto
4   desconhecida  baixa   nenhuma  acima_35  baixo
5   desconhecida  baixa  adequada  acima_35  baixo
6           ruim  baixa   nenhuma      0_15   alto
8            boa  baixa   nenhuma  acima_35  baixo
9            boa   alta  adequada  acima_35  baixo
10           boa   alta   nenhuma      0_15   alto
12           boa   alta   nenhuma  acima_35  baixo
13          ruim   alta   nenhuma     15_35   alto

Total de registros: 11

X codificado:
[[2 0 1 0]
 [1 0 1 1]
 [1 1 1 2]
 [1 1 1 2]
 [1 1 0 2]
 [2 1 1 0]
 [0 1 1 2]
 [0 0 0 2]
 [0 0 1 0]
 [0 0 1 2]
 [2 0 1 1]]

y codificado:
[0 0 0 1 1 0 1 1 0 1 0]

Classes: ['alto' 'baixo']


## 3. Treinar o algoritmo de Regressão Logística

In [23]:
# Treinar o modelo de regressão logística
reglog_risco_credito = LogisticRegression(random_state=1)
reglog_risco_credito.fit(X_encoded, y_encoded)

print("Modelo de Regressão Logística treinado com sucesso!")

Modelo de Regressão Logística treinado com sucesso!


## 4. Verificar o intercepto (B0)

In [24]:
print(f"Intercepto (B0): {reglog_risco_credito.intercept_[0]}")

Intercepto (B0): -0.8085515186482205


## 5. Verificar os coeficientes

In [25]:
print(f"Coeficientes: {reglog_risco_credito.coef_}")

Coeficientes: [[-0.76706797  0.23921064 -0.47989768  1.12196145]]


## 6. Fazer predições com os exemplos

In [26]:
# Exemplo a) história boa, dívida alta, garantias nenhuma, renda > 35
exemplo_a = [
    label_encoders[0].transform(['boa'])[0],
    label_encoders[1].transform(['alta'])[0],
    label_encoders[2].transform(['nenhuma'])[0],
    label_encoders[3].transform(['acima_35'])[0]
]

# Exemplo b) história ruim, dívida alta, garantias adequada, renda < 15
exemplo_b = [
    label_encoders[0].transform(['ruim'])[0],
    label_encoders[1].transform(['alta'])[0],
    label_encoders[2].transform(['adequada'])[0],
    label_encoders[3].transform(['0_15'])[0]
]

# Fazer predições
pred_a = reglog_risco_credito.predict([exemplo_a])[0]
pred_b = reglog_risco_credito.predict([exemplo_b])[0]

# Converter para labels originais
classe_a = le_y.inverse_transform([pred_a])[0]
classe_b = le_y.inverse_transform([pred_b])[0]

print(f"\nPredição exemplo a) história boa, dívida alta, garantias nenhuma, renda > 35: {classe_a}")
print(f"Predição exemplo b) história ruim, dívida alta, garantias adequada, renda < 15: {classe_b}")


Predição exemplo a) história boa, dívida alta, garantias nenhuma, renda > 35: baixo
Predição exemplo b) história ruim, dívida alta, garantias adequada, renda < 15: alto


##Algoritmo de Regressão Logística para uma base de dados maior (Credit Data)

7. Agora aplique a Regressão Logística na base de dados ‘credit.pkl’. De quanto foi a taxa de acerto?

8. O resultado com a base de dados ‘credit.pkl’ é melhor que os resultados do Naive Bayes e das Florestas Aleatórias? Descreva sua análise de resultados (observe que para isso você deverá visualizar os resultados da Matriz de Confusão, acurácia, precisão e recall).

#PARTE 4: SVM

1. Aplique o algoritmo SVM na base de dados ‘credit.pkl’.
2. Inicialmente treine o SVM com kernel linear, valor do parâmetro C = 1.0 e ‘random_state =1’
3. Utilize o comando do sklearn accuray_score para calcular a acurácia do seu algoritmo. O resultado deve ser 0.946
4. Teste os demais kernels e anote os resultados. Qual o melhor kernel para a sua base de dados?
    * Polinomial
    * Sigmoide
    * rbf
5. Aumente o valor do parâmetro C aplicado ao melhor kernel e verifique se há mudanças no resultado do seu SVM.
6. O Grid Search (pesquisa em grade) é uma técnica utilizada para melhorar a precisão e a generalização dos modelos de aprendizado de máquina. Ela é usada para realizar ajustes de hiperparâmetros durante o treinamento de um modelo. O grid search automatiza o processo de encontrar hiperparâmetros ideais, economizando esforço humano em comparação com o ajuste manual, mas pode até ser mais custoso do ponto de vista de desempenho, pois testa todas as combinações possíveis e retorna a que obteve melhor desempenho.
Agora, aplique o GridSearch do Scikit-Learn (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) para ajudar a identificar os melhores hiperparâmetros para o seu modelo. Qual foi a melhor combinação de hiperparâmetros encontrada? O modelo com melhor desempenho foi obtido com os parâmetros ajustados manualmente ou com o GridSearch?

In [27]:
from sklearn.svm import SVC

## Base de Dados Credit Data

# Análise dos resultados dos 4 algoritmos utilizados:

6. O resultado do SVM é melhor que os resultados do Naive Bayes, Florestas Aleatórias e Regressão Logística? Descreva sua análise de resultados (observe que para isso você deverá visualizar os resultados da Matriz de Confusão, acurácia, precisão e recall).

# Conclusão
# Descreva aqui suas conclusões sobre os resultados obtidos com os diferentes algoritmos e a importância da escolha adequada do modelo e hiperparâmetros.
